In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import DeepFool
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np

from sklearn import metrics

import joblib

2024-02-26 02:19:58.054251: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 02:19:58.057040: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 02:19:58.088695: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 02:19:58.088738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 02:19:58.090358: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_test.npy")
x_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_train.npy")
x_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_val.npy")
y_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_test.npy")
y_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_train.npy")
y_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_val.npy")

In [4]:
model = joblib.load("/home/jovyan/X-IIoT/dl/dl_new/model/dnn.joblib")

2024-02-26 02:20:03.411762: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://961f7369c02d4a7395453111879dce0b: INVALID_ARGUMENT: ram://961f7369c02d4a7395453111879dce0b is a directory.


In [5]:
x_test_trimmed = x_test[20000:25000]
y_test_trimmed = y_test[20000:25000]

In [7]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=19,
    input_shape=(58,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [8]:
attack = DeepFool(classifier=classifier, epsilon=0.01)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

DeepFool:   0%|          | 0/5000 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 5000/5000 [1:16:30<00:00,  1.09it/s]  


In [9]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

157/157 [==============================] - 0s 575us/step
Accuracy: 0.136
AUC: 0.6102342186627555

macro
Precision: 0.09876952007417407
Recall: 0.1144576552780136
F1 Score: 0.09294755559793225

weighted
Precision: 0.2679652931290648
Recall: 0.136
F1 Score: 0.16674845625723037

Mean FNR: 0.8855423447219865
Mean TNR: 0.942622408046837
Mean FPR: 0.057377591953163096
Mean TPR: 0.1144576552780136


In [10]:
attack = DeepFool(classifier=classifier, epsilon=0.05)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

DeepFool: 100%|██████████| 5000/5000 [1:13:50<00:00,  1.13it/s]


In [11]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

157/157 [==============================] - 0s 609us/step
Accuracy: 0.1416
AUC: 0.6082219391703667

macro
Precision: 0.10000115137799186
Recall: 0.11496494760521714
F1 Score: 0.093707313524651

weighted
Precision: 0.27542092412333563
Recall: 0.1416
F1 Score: 0.1735824857186065

Mean FNR: 0.8850350523947828
Mean TNR: 0.9429489857907156
Mean FPR: 0.05705101420928448
Mean TPR: 0.11496494760521714


In [11]:
attack = DeepFool(classifier=classifier, epsilon=0.1)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

DeepFool: 100%|██████████| 10000/10000 [4:09:54<00:00,  1.50s/it]  


In [12]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

313/313 [==============================] - 0s 589us/step
Accuracy: 0.1537
AUC: 0.6231929064660084

macro
Precision: 0.06853119441323076
Recall: 0.07050011978965631
F1 Score: 0.0543755471725466

weighted
Precision: 0.23125504930524404
Recall: 0.1537
F1 Score: 0.18045331338935855

Mean FNR: 0.9294998802103438
Mean TNR: 0.9365581719746082
Mean FPR: 0.06344182802539171
Mean TPR: 0.07050011978965631


In [9]:
attack = DeepFool(classifier=classifier, epsilon=0.15)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

DeepFool:   0%|          | 0/5000 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 5000/5000 [2:04:45<00:00,  1.50s/it]  


In [10]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

157/157 [==============================] - 0s 610us/step
Accuracy: 0.1556
AUC: 0.6188928543332562

macro
Precision: 0.06924474679269647
Recall: 0.07098424545367793
F1 Score: 0.05821787053637322

weighted
Precision: 0.23225553788236517
Recall: 0.1556
F1 Score: 0.18276870486718763

Mean FNR: 0.9290157545463221
Mean TNR: 0.9362994691867641
Mean FPR: 0.06370053081323586
Mean TPR: 0.07098424545367793


In [11]:
attack = DeepFool(classifier=classifier, epsilon=0.2)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

DeepFool: 100%|██████████| 5000/5000 [2:05:55<00:00,  1.51s/it]  


In [12]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

157/157 [==============================] - 0s 592us/step
Accuracy: 0.1564
AUC: 0.6183252345763915

macro
Precision: 0.07086415565830172
Recall: 0.07105671578613558
F1 Score: 0.059342273925384804

weighted
Precision: 0.23333992126875583
Recall: 0.1564
F1 Score: 0.18371389976001073

Mean FNR: 0.9289432842138644
Mean TNR: 0.9363657367997887
Mean FPR: 0.06363426320021108
Mean TPR: 0.07105671578613558


In [13]:
attack = DeepFool(classifier=classifier, epsilon=0.25)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

DeepFool: 100%|██████████| 5000/5000 [2:06:05<00:00,  1.51s/it]  


In [14]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

157/157 [==============================] - 0s 610us/step
Accuracy: 0.157
AUC: 0.6176480906894147

macro
Precision: 0.07059996780323768
Recall: 0.0703946114908643
F1 Score: 0.05894065489672211

weighted
Precision: 0.2342163857459694
Recall: 0.157
F1 Score: 0.18456470494352548

Mean FNR: 0.9296053885091358
Mean TNR: 0.9363982972922743
Mean FPR: 0.06360170270772564
Mean TPR: 0.0703946114908643


In [15]:
attack = DeepFool(classifier=classifier, epsilon=0.3)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

DeepFool: 100%|██████████| 5000/5000 [2:06:20<00:00,  1.52s/it]  


In [16]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

157/157 [==============================] - 0s 602us/step
Accuracy: 0.1586
AUC: 0.6170524100501221

macro
Precision: 0.07321796618979222
Recall: 0.0705395521557796
F1 Score: 0.05919346805286544

weighted
Precision: 0.23564112057949604
Recall: 0.1586
F1 Score: 0.18609323723678212

Mean FNR: 0.9294604478442204
Mean TNR: 0.9364325198307268
Mean FPR: 0.06356748016927304
Mean TPR: 0.0705395521557796


In [ ]:
attack = DeepFool(classifier=classifier, epsilon=0.01)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

DeepFool:   0%|          | 0/5000 [00:00<?, ?it/s]

DeepFool:   3%|▎         | 132/5000 [03:25<4:04:30,  3.01s/it]

In [ ]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

In [ ]:
attack = DeepFool(classifier=classifier, epsilon=0.05)

x_test_adv = attack.generate(x=x_test_trimmed)  # x_test is your MNIST test images

In [ ]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)